# Importações


In [154]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from fuzzywuzzy import process
from enum import Enum


FERIADOS = pd.read_csv("../assets/feriados.csv")

### Types


In [155]:
# cSpell: disable
class BSColorsEnum(Enum):

    DANGER_COLOR = "#dc3545"

    WARNING_COLOR = "#ffc107"

    SUCCESS_COLOR = "#198754"

    GREY_500_COLOR = "#adb5bd"

    GREY_600_COLOR = "#6c757d"

    GREY_700_COLOR = "#495057"

    GREY_800_COLOR = "#343a40"

    GREY_900_COLOR = "#212529"

    PRIMARY_COLOR = "#0d6efd"

    SECONDARY_COLOR = "#6c757d"

    INFO_COLOR = "#0dcaf0"

    GRAY_COLOR = "#adb5bd"

    TEAL_COLOR = "#20c997"

    ORANGE_COLOR = "#fd7e14"

    INDIGO_COLOR = "#6610f2"

    PINK_COLOR = "#d63384"

    PURPLE_COLOR = "#6f42c1"

    GREY_400_COLOR = "#ced4da"

    SPACE_CADET_COLOR = "#282f44"

    BLUE_DELFT_COLOR = "#0d6efd"


class IndicatorType(Enum):
    PERFORMANCE = "performance"
    REPAIR = "reparo"
    EFFICIENCY = "eficiencia"


def get_color(value, max_value):
    """
    Retorna uma cor hexadecimal com base no valor fornecido e no valor máximo.

    Parâmetros:
    value (float): O valor para o qual a cor será calculada.
    max_value (float): O valor máximo possível.

    Retorna:
    str: Uma cor hexadecimal correspondente ao valor fornecido.
    """

    # Cria um mapa de cores que vai do vermelho ao verde
    cmap = plt.get_cmap("RdYlGn")

    # Normaliza o valor para um número entre 0 e 1
    normalized_value = float(value) / max_value

    # Obtém a cor correspondente do mapa de cores
    rgba_color = cmap(normalized_value)

    # Converte a cor RGBA para uma string de cor hexadecimal
    hex_color = (
        f"#{int(rgba_color[0]*255):02x}{int(rgba_color[1]*255):02x}{int(rgba_color[2]*255):02x}"
    )

    return hex_color

# Database


## Conexão com o banco de dados


In [156]:
# database/connection.py

# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = getenv("PYMSSQL_USER")
        self.__password = getenv("PYMSSQL_PASSWORD")
        self.__database = getenv("PYMSSQL_DATABASE_AUTOMACAO")
        self.__database_totvsdb = getenv("PYMSSQL_DATABASE_TOTVSDB")
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = getenv("PYMSSQL_SERVER")

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def get_connection_totvsdb(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database_totvsdb};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            conexao_totvsdb = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", pool_size=2)
            return conexao_totvsdb
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

## Leitura do banco de dados


In [157]:
# database/db_read.py


# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_automacao_query(self, table: str, where: str = None, orderby: str = None) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

    def get_totvsdb_data(self, query: str) -> pd.DataFrame:
        """
        Retrieves data from the TotvsDB database using the provided SQL query.

        Args:
            query (str): The SQL query to execute.

        Returns:
            pd.DataFrame: A pandas DataFrame containing the retrieved data.

        Raises:
            Exception: If an error occurs while retrieving the data.

        """
        connection = None
        try:
            connection = self.get_connection_totvsdb()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as e:
            print(f"Erro ao buscar dados: {e}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_totvsdb_query(
        self, select: str, table: str, join: str = None, where: str = None, orderby: str = None
    ) -> str:
        """
        Creates a SQL query string for querying the TOTVS database.

        Args:
            select (str): The SELECT clause of the query.
            table (str): The table name to query.
            join (str, optional): The JOIN clause of the query. Defaults to None.
            where (str, optional): The WHERE clause of the query. Defaults to None.
            orderby (str, optional): The ORDER BY clause of the query. Defaults to None.

        Returns:
            str: The SQL query string.
        """
        query = f"SELECT {select} FROM {table}"

        if join:
            query += f" {join}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

## Query para o banco de dados


In [158]:
# // database/get_data.py
# cSpell: disable=invalid-name
class GetData:
    """
    Essa classe é responsável por realizar a leitura dos dados do banco de dados.
    É utilizada para fazer a leitura em segundo plano, sem que o usuário perceba.
    """

    def __init__(self):
        self.db_read = Read()

    def get_data(self) -> tuple:
        """
        Realiza a leitura dos dados do banco de dados.
        Retorna na ordem: df_ihm, df_info, df_cadastro
        """

        # Dia de hoje
        now = pd.to_datetime("today")

        # Encontrando primeiro dia do mês atual
        first_day = now.replace(day=1)

        # Mantendo apenas a data
        first_day = first_day.strftime("%Y-%m-%d")

        # Query para leitura dos dados de ocorrência
        query_ihm = self.db_read.create_automacao_query(
            table="maquina_ihm",
            where=f"data_registro >= '{first_day}'",
        )

        query_info = (
            "SELECT"
            " t1.maquina_id,"
            " (SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha,"
            " (SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica,"
            " t1.status,"
            " t1.turno,"
            " t1.contagem_total_ciclos,"
            " t1.contagem_total_produzido,"
            " t1.data_registro,"
            " t1.hora_registro"
            " FROM "
            " AUTOMACAO.dbo.maquina_info t1"
            f" WHERE data_registro >= '{first_day}'"
            " ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
        )

        query_production = (
            "SELECT * "
            "FROM ( "
            "SELECT "
            "(SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica, "
            "(SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha, "
            "t1.maquina_id, "
            "t1.turno, "
            "t1.status, "
            "t1.contagem_total_ciclos as total_ciclos, "
            "t1.contagem_total_produzido as total_produzido, "
            "t1.data_registro, "
            "t1.hora_registro, "
            "ROW_NUMBER() OVER ( "
            "PARTITION BY t1.data_registro, t1.turno, t1.maquina_id "
            "ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
            ") AS rn "
            "FROM AUTOMACAO.dbo.maquina_info t1 "
            ") AS t "
            f" WHERE rn = 1 AND data_registro >= '{first_day}' AND hora_registro > '00:01'"
            " ORDER BY data_registro DESC, linha"
        )

        # Leitura dos dados
        df_ihm = self.db_read.get_automacao_data(query_ihm)
        df_info = self.db_read.get_automacao_data(query_info)
        df_info_production = self.db_read.get_automacao_data(query_production)

        # Verificando se os dados foram lidos corretamente
        if df_ihm.empty or df_info.empty or df_info_production.empty:
            print("====== Erro na leitura dos dados ======")
            return None, None, None

        return df_ihm, df_info, df_info_production


get_data = GetData()
df_ihm, df_info, df_info_production = get_data.get_data()

## Testes de saída do banco de dados


In [159]:
df_ihm.head(20)

,recno,linha,maquina_id,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro,hora_registro
0,15,1,TMF003,Parada Programada,,Parada Planejada,Sem Produção,,002357,2024-05-01,11:28:20.666666
1,16,1,TMF003,Parada Programada,,Parada Planejada,Backup,,002357,2024-05-01,06:47:36.466666
2,18,1,TMF003,Parada Programada,,Parada Planejada,Refeição,,002357,2024-05-03,08:29:24.796666
3,20,2,TMF011,Parada Programada,,Parada Planejada,Sem Produção,,002357,2024-05-03,09:08:52.143333
4,22,1,TMF003,Manutenção,Termoformadora,Bandeja Murcha ou Cheia,Realizar análise de falha,015360,002357,2024-05-03,08:55:55.110000
5,23,2,TMF011,Manutenção,Termoformadora,Bandeja deformada ou manchada,Realizar análise de falha,015360,002357,2024-05-03,07:56:50.340000
6,24,2,TMF011,Fluxo,Termoformadora,Esteira Cheia,Robô parado,,002357,2024-05-03,08:57:06.950000
7,25,1,TMF003,Limpeza,,Limpeza para parada de Fábrica,,,002357,2024-05-03,08:37:53.763333
8,26,1,TMF003,Parada Programada,,Parada Planejada,Backup,,002357,2024-05-03,08:58:06.920000


In [160]:
df_info.head(20)

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF004,9,1,false,MAT,2440.0,2354.0,2024-05-03,11:25:25.100000
1,TMF014,8,1,true,MAT,3016.0,3212.0,2024-05-03,11:25:24.103333
2,TMF006,7,1,false,MAT,3110.0,3094.0,2024-05-03,11:25:23.100000
3,TMF001,6,1,true,MAT,4248.0,4150.0,2024-05-03,11:25:22.096666
4,TMF002,5,1,true,MAT,4152.0,3904.0,2024-05-03,11:25:21.096666
5,TMF012,4,1,false,MAT,2268.0,2210.0,2024-05-03,11:25:20.096666
6,TMF015,0,,false,MAT,140.0,134.0,2024-05-03,11:25:19.093333
7,TMF011,2,1,false,MAT,2.0,1490.0,2024-05-03,11:25:18.090000
8,TMF003,1,1,false,MAT,1922.0,1898.0,2024-05-03,11:25:17.090000
9,TMF009,14,2,false,MAT,0.0,0.0,2024-05-03,11:24:40.896666


In [161]:
df_info_production.head(20)

,fabrica,linha,maquina_id,turno,status,total_ciclos,total_produzido,data_registro,hora_registro,rn
0,,0,TMF015,MAT,false,140.0,134.0,2024-05-03,11:25:19.093333,1
1,,0,TMF015,NOT,false,140.0,134.0,2024-05-03,07:59:18.500000,1
2,1,1,TMF003,MAT,false,1922.0,1898.0,2024-05-03,11:25:17.090000,1
3,1,1,TMF003,NOT,true,8728.0,8712.0,2024-05-03,07:59:16.500000,1
4,1,2,TMF011,MAT,false,2.0,1490.0,2024-05-03,11:25:18.090000,1
5,1,2,TMF011,NOT,false,1624.0,1490.0,2024-05-03,07:59:17.496666,1
6,1,4,TMF012,MAT,false,2268.0,2210.0,2024-05-03,11:25:20.096666,1
7,1,4,TMF012,NOT,true,8004.0,7910.0,2024-05-03,07:59:19.500000,1
8,1,5,TMF002,MAT,true,4152.0,3904.0,2024-05-03,11:25:21.096666,1
9,1,5,TMF002,NOT,true,6900.0,6334.0,2024-05-03,07:59:20.503333,1


# Limpeza de dados e análise exploratória


## Análise de dados - Clean Data


In [162]:
# service/clean_data.py


# cSpell: disable=invalid-name
class CleanData:
    """
    Essa classe é responsável por limpar os dados lidos do banco de dados.
    """

    def __init__(self, df_ihm, df_info, df_info_production):
        self.df_ihm = df_ihm
        self.df_info = df_info
        self.df_info_production = df_info_production

    def __clean_basics(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados básicos de um DataFrame.
        """

        # Remove valores duplicados
        df = df.drop_duplicates()

        # Remove linhas com valores nulos
        df = df.dropna(subset=["maquina_id", "data_registro", "hora_registro"])

        # Converte a coluna 'hora_registro' para string e remove os milissegundos
        df["hora_registro"] = df["hora_registro"].astype(str).str.split(".").str[0]

        # Converte as colunas 'data_registro' e 'hora_registro' para datetime
        df["data_registro"] = pd.to_datetime(df["data_registro"])
        df["hora_registro"] = pd.to_datetime(df["hora_registro"], format="%H:%M:%S").dt.time

        # Substitui os valores NaN por 0 antes de converter para int
        df["linha"] = df["linha"].fillna(0).astype(int)

        # Coluna linha para int
        df["linha"] = df["linha"].astype(int)

        # Remover onde a 'linha' é 0
        df = df[df["linha"] != 0]

        return df

    def __clean_info(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de informações.
        Fica desatualizado após implementação das IHMs.
        Manter apenas para código legado.
        """

        # Reordenar o dataframe
        df = df.sort_values(by=["maquina_id", "data_registro", "hora_registro"])

        # Remover a primeira entrada caso seja do turno "VES"
        mask = (df["turno"] == "VES") & (df["maquina_id"] != df["maquina_id"].shift())
        df = df[~mask]

        # Ajustar o turno VES caso a hora da entrada passe de 00:00, para 23:59 do dia anterior
        mask = (
            (df["turno"] == "VES")
            & (df["hora_registro"] > time(0, 0, 0))
            & (df["hora_registro"] < time(0, 5, 0))
        )

        df.loc[mask, "hora_registro"] = time(23, 59, 59)
        df.loc[mask, "data_registro"] = df.loc[mask, "data_registro"] - timedelta(days=1)

        # Reordenar o dataframe
        df = df.sort_values(by=["linha", "data_registro", "hora_registro"])

        # Reiniciar o índice
        df = df.reset_index(drop=True)

        return df

    def __clean_prod(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de produção.
        """

        # Coluna auxiliar para ordenar os turnos
        df["turno_aux"] = df["turno"].map({"MAT": 2, "VES": 3, "NOT": 1})

        # Ordenar os valores
        df = df.sort_values(by=["linha", "data_registro", "turno_aux"])

        # Remover a coluna auxiliar
        df = df.drop(columns=["turno_aux", "rn", "status"])

        # Coluna linha para int
        df["linha"] = df["linha"].astype(int)

        # Reiniciar o índice
        df = df.reset_index(drop=True)

        return df

    def clean_data(self) -> tuple:
        """
        Limpa os dados lidos do banco de dados.
        Retorna na ordem: df_ihm, df_info, df_info_production
        """

        # Limpando os dados(removendo linhas duplicadas ou nulas em informações importantes)
        self.df_ihm = self.__clean_basics(self.df_ihm)
        self.df_info = self.__clean_basics(self.df_info)
        self.df_info_production = self.__clean_basics(self.df_info_production)

        # Ajustando o status
        # Ajusta nomenclatura do status
        self.df_info["status"] = np.where(self.df_info["status"] == "true", "rodando", "parada")

        # Ajustes nas informações de produção
        self.df_info_production = self.__clean_prod(self.df_info_production)

        # NOTE: A fonção abaixo ficará desatualizada após a implementação das IHMs
        self.df_info = self.__clean_info(self.df_info)

        return self.df_ihm, self.df_info, self.df_info_production


clean_data = CleanData(df_ihm.copy(), df_info.copy(), df_info_production.copy())
df_ihm_cleaned, df_info_cleaned, df_info_production_cleaned = clean_data.clean_data()

### Saída de dados limpos


In [163]:
df_ihm_cleaned

,recno,linha,maquina_id,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro,hora_registro
0,15,1,TMF003,Parada Programada,,Parada Planejada,Sem Produção,,002357,2024-05-01,11:28:20
1,16,1,TMF003,Parada Programada,,Parada Planejada,Backup,,002357,2024-05-01,06:47:36
2,18,1,TMF003,Parada Programada,,Parada Planejada,Refeição,,002357,2024-05-03,08:29:24
3,20,2,TMF011,Parada Programada,,Parada Planejada,Sem Produção,,002357,2024-05-03,09:08:52
4,22,1,TMF003,Manutenção,Termoformadora,Bandeja Murcha ou Cheia,Realizar análise de falha,015360,002357,2024-05-03,08:55:55
5,23,2,TMF011,Manutenção,Termoformadora,Bandeja deformada ou manchada,Realizar análise de falha,015360,002357,2024-05-03,07:56:50
6,24,2,TMF011,Fluxo,Termoformadora,Esteira Cheia,Robô parado,,002357,2024-05-03,08:57:06
7,25,1,TMF003,Limpeza,,Limpeza para parada de Fábrica,,,002357,2024-05-03,08:37:53
8,26,1,TMF003,Parada Programada,,Parada Planejada,Backup,,002357,2024-05-03,08:58:06


In [164]:
df_info_cleaned

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF003,1,1,parada,NOT,18.0,12.0,2024-05-01,00:01:06
1,TMF003,1,1,parada,NOT,18.0,12.0,2024-05-01,00:03:06
2,TMF003,1,1,parada,NOT,18.0,12.0,2024-05-01,00:05:06
3,TMF003,1,1,parada,NOT,18.0,12.0,2024-05-01,00:07:06
4,TMF003,1,1,parada,NOT,18.0,12.0,2024-05-01,00:09:06
...,...,...,...,...,...,...,...,...,...
19856,TMF009,14,2,parada,MAT,0.0,0.0,2024-05-03,11:16:40
19857,TMF009,14,2,parada,MAT,0.0,0.0,2024-05-03,11:18:40
19858,TMF009,14,2,parada,MAT,0.0,0.0,2024-05-03,11:20:40
19859,TMF009,14,2,parada,MAT,0.0,0.0,2024-05-03,11:22:40


In [165]:
df_info_production_cleaned

,fabrica,linha,maquina_id,turno,total_ciclos,total_produzido,data_registro,hora_registro
0,1,1,TMF003,NOT,18.0,12.0,2024-05-01,07:59:08
1,1,1,TMF003,MAT,18.0,12.0,2024-05-01,15:59:09
2,1,1,TMF003,VES,72.0,56.0,2024-05-01,23:59:10
3,1,1,TMF003,NOT,6398.0,6274.0,2024-05-02,07:59:12
4,1,1,TMF003,MAT,2124.0,2116.0,2024-05-02,15:59:13
...,...,...,...,...,...,...,...,...
89,2,14,TMF009,NOT,0.0,0.0,2024-05-02,07:58:10
90,2,14,TMF009,MAT,0.0,0.0,2024-05-02,15:58:11
91,2,14,TMF009,VES,0.0,0.0,2024-05-02,23:58:38
92,2,14,TMF009,NOT,0.0,0.0,2024-05-03,07:58:40


# Unindo os dados


## IHM + Info


In [166]:
class JoinInfoIHM:
    """
    Essa classe é responsável por juntar os DataFrames de informações e IHM.
    """

    def __init__(self, df_ihm, df_info):
        self.df_ihm = df_ihm
        self.df_info = df_info

    def join_data(self) -> pd.DataFrame:
        """
        Junta os DataFrames de informações e IHM.
        """

        # Cria uma coluna auxiliar com data e hora para fazer o merge
        self.df_info["data_hora"] = pd.to_datetime(
            self.df_info["data_registro"].astype(str)
            + " "
            + self.df_info["hora_registro"].astype(str)
        )

        self.df_ihm["data_hora"] = pd.to_datetime(
            self.df_ihm["data_registro"].astype(str)
            + " "
            + self.df_ihm["hora_registro"].astype(str)
        )

        # Classifica os DataFrames
        self.df_info = self.df_info.sort_values(by="data_hora")
        self.df_ihm = self.df_ihm.sort_values(by="data_hora")

        # Junta os DataFrames
        df = pd.merge_asof(
            self.df_info,
            self.df_ihm.drop(columns="recno"),
            on="data_hora",
            by="maquina_id",
            direction="nearest",
            tolerance=pd.Timedelta("1 min 10 s"),
        )

        # Ajustas os tipos das colunas contagem_total_ciclos e contagem_total_produzido para int
        df["contagem_total_ciclos"] = df["contagem_total_ciclos"].astype("Int64")
        df["contagem_total_produzido"] = df["contagem_total_produzido"].astype("Int64")

        # Reordenar as colunas mantendo só as colunas necessárias e renomeando as colunas para facilitar a compreensão
        df = df[
            [
                "fabrica",
                "linha_x",
                "maquina_id",
                "turno",
                "status",
                "contagem_total_ciclos",
                "contagem_total_produzido",
                "data_registro_x",
                "hora_registro_x",
                "motivo",
                "equipamento",
                "problema",
                "causa",
                "os_numero",
                "operador_id",
                "data_registro_y",
                "hora_registro_y",
            ]
        ]

        df.columns = [
            "fabrica",
            "linha",
            "maquina_id",
            "turno",
            "status",
            "contagem_total_ciclos",
            "contagem_total_produzido",
            "data_registro",
            "hora_registro",
            "motivo",
            "equipamento",
            "problema",
            "causa",
            "os_numero",
            "operador_id",
            "data_registro_ihm",
            "hora_registro_ihm",
        ]

        # Reordenar pela linha, data_registro_info e hora_registro_info
        df = df.sort_values(by=["linha", "data_registro", "hora_registro"])

        return df


join_info_ihm = JoinInfoIHM(df_ihm_cleaned.copy(), df_info_cleaned.copy())
df_joined = join_info_ihm.join_data()

### Saída de Join


In [167]:
df_joined

,fabrica,linha,maquina_id,turno,status,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro_ihm,hora_registro_ihm
0,1,1,TMF003,NOT,parada,18,12,2024-05-01,00:01:06,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,1,1,TMF003,NOT,parada,18,12,2024-05-01,00:03:06,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
16,1,1,TMF003,NOT,parada,18,12,2024-05-01,00:05:06,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
24,1,1,TMF003,NOT,parada,18,12,2024-05-01,00:07:06,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
32,1,1,TMF003,NOT,parada,18,12,2024-05-01,00:09:06,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,2,14,TMF009,MAT,parada,0,0,2024-05-03,11:16:40,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
19813,2,14,TMF009,MAT,parada,0,0,2024-05-03,11:18:40,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
19826,2,14,TMF009,MAT,parada,0,0,2024-05-03,11:20:40,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
19839,2,14,TMF009,MAT,parada,0,0,2024-05-03,11:22:40,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


# Service


## Info + IHM Service


In [168]:
class ServiceInfoIHM:
    """
    Essa classe é responsável por realizar a leitura, limpeza e junção dos dados de informações e IHM.
    """

    def __init__(self):
        self.get_data = GetData()
        self.clean_data = CleanData
        self.join_info_ihm = JoinInfoIHM
        self.data = self.get_data.get_data()

    def __get_info_ihm(self) -> pd.DataFrame:
        """
        Realiza a leitura, limpeza e junção dos dados de informações e IHM.
        """

        # Realiza a leitura dos dados
        df_ihm, df_info, _ = self.data

        # Verifica se os dados foram lidos corretamente
        if df_ihm is None or df_info is None or df_info_production is None:
            return None

        # Limpa os dados
        clean_data = self.clean_data(df_ihm, df_info, df_info_production)
        df_ihm_cleaned, df_info_cleaned, _ = clean_data.clean_data()

        # Junta os dados
        join_info_ihm = self.join_info_ihm(df_ihm_cleaned, df_info_cleaned)
        df_joined = join_info_ihm.join_data()

        return df_joined

    def __identify_changes(self, df, column) -> pd.Series:
        return df[column].ne(df[column].shift())

    def __status_change(self, df):
        # Checa se houve mudança de status, maquina_id e turno
        columns_to_check = ["status", "maquina_id", "turno"]
        for column in columns_to_check:
            df[f"{column}_change"] = self.__identify_changes(df, column)

        # Coluna auxiliar que identifica se houve alguma mudança
        df["change"] = df[["status_change", "maquina_id_change", "turno_change"]].any(axis=1)

        return df

    def __fill_occ(self, df):
        # Preenche os valores nulos de paradas
        df["motivo"] = df.groupby("group")["motivo"].transform(lambda x: x.ffill().bfill())
        df["equipamento"] = df.groupby("group")["equipamento"].transform(
            lambda x: x.ffill().bfill()
        )
        df["problema"] = df.groupby("group")["problema"].transform(lambda x: x.ffill().bfill())
        df["causa"] = df.groupby("group")["causa"].transform(lambda x: x.ffill().bfill())
        df["os_numero"] = df.groupby("group")["os_numero"].transform(lambda x: x.ffill().bfill())
        df["operador_id"] = df.groupby("group")["operador_id"].transform(
            lambda x: x.ffill().bfill()
        )
        df["data_registro_ihm"] = df.groupby("group")["data_registro_ihm"].transform(
            lambda x: x.ffill().bfill()
        )
        df["hora_registro_ihm"] = df.groupby("group")["hora_registro_ihm"].transform(
            lambda x: x.ffill().bfill()
        )

        # Se os dado de uma coluna for '' ou ' ', substituir por NaN
        df = df.replace(r"^s*$", None, regex=True)
        # O ^ indica o início de uma string, o $ indica o fim de uma string, e s* zero ou mais espaços em branco

        return df

    def __group_and_calc_time(self, df):
        # Agrupa as mudanças
        df = (
            df.groupby(["group"])
            .agg(
                fabrica=("fabrica", "first"),
                linha=("linha", "first"),
                maquina_id=("maquina_id", "first"),
                turno=("turno", "first"),
                status=("status", "first"),
                data_registro=("data_registro", "first"),
                hora_registro=("hora_registro", "first"),
                motivo=("motivo", "first"),
                equipamento=("equipamento", "first"),
                problema=("problema", "first"),
                causa=("causa", "first"),
                os_numero=("os_numero", "first"),
                operador_id=("operador_id", "first"),
                data_registro_ihm=("data_registro_ihm", "first"),
                hora_registro_ihm=("hora_registro_ihm", "first"),
                data_hora=("data_hora", "first"),
                change=("change", "first"),
                maquina_id_change=("maquina_id_change", "first"),
                change_date=("change_date", "first"),
                motivo_change=("motivo_change", "first"),
            )
            .reset_index()
        )

        # Nova coluna com a data_hora_final do status/parada
        df["data_hora_final"] = (
            df.groupby("maquina_id")["data_hora"].shift(-1).where(~df["maquina_id_change"])
        )

        # Atualiza a hora final caso mude a máquina
        mask = df["maquina_id_change"]
        df["data_hora_final"] = np.where(
            mask,
            df["change_date"].shift(-1),
            df["data_hora_final"],
        )

        # Caso a data_hora_final seja nula, remove a linha
        df = df.dropna(subset=["data_hora_final"]).reset_index(drop=True)

        # Calcula o tempo de cada status
        df["tempo"] = (
            pd.to_datetime(df["data_hora_final"]) - pd.to_datetime(df["data_hora"])
        ).dt.total_seconds() / 60

        # Arredondar e converter para inteiro
        df["tempo"] = df["tempo"].round().astype(int)

        return df

    def get_info_ihm_adjusted(self) -> pd.DataFrame:
        """
        Realiza a leitura, limpeza e junção dos dados de informações e IHM.
        Ajusta os dados para o formato correto.
        """

        # Realiza a leitura, limpeza e junção dos dados
        df_joined = self.__get_info_ihm()

        if df_joined is None:
            return None

        # ========================= Identifica Onde Há Mudança De Status ========================= #

        df_joined = self.__status_change(df_joined)

        # ========================= Preenchendo Valores Nulos De Paradas ========================= #

        # Cria um grupo para cada mudança de status
        df_joined["group"] = df_joined["change"].cumsum()

        # Preenche os valores nulos de paradas
        df_joined = self.__fill_occ(df_joined)

        # Coluna auxiliar para identificar mudança na coluna motivo, se não for nula
        mask = (df_joined["motivo"].ne(df_joined["motivo"].shift())) | (
            df_joined["causa"].ne(df_joined["causa"].shift())
        )
        df_joined["motivo_change"] = mask & df_joined["motivo"].notnull()

        # Atualiza o change
        df_joined["change"] = df_joined["change"] | df_joined["motivo_change"]

        # Refaz o grupo para considerar a mudança na coluna motivo
        df_joined["group"] = df_joined["change"].cumsum()

        # Coluna auxiliar para unir data e hora
        df_joined["data_hora"] = pd.to_datetime(
            df_joined["data_registro"].astype(str) + " " + df_joined["hora_registro"].astype(str)
        )

        # Coluna auxiliar para identificar a data/hora da mudança
        df_joined["change_date"] = (
            df_joined.groupby("maquina_id")["data_hora"].shift(0).where(df_joined["change"])
        )

        # ============================ Calcula O Tempo Parada Ou Rodando ========================== #

        # Calcula os tempos
        df_joined = self.__group_and_calc_time(df_joined)

        # Ajustar status para levar em conta testes
        mask = (
            (df_joined["status"] == "rodando")
            & (df_joined["tempo"] < 10)
            & (df_joined["turno"].eq(df_joined["turno"].shift(-1)))
        )

        df_joined["status"] = np.where(mask, "parada", df_joined["status"])

        # Ajuste em change para refletir alterações
        df_joined = self.__status_change(df_joined)
        df_joined["change"] = df_joined["change"] | df_joined["motivo_change"]

        # Refaz o group
        df_joined["group"] = df_joined["change"].cumsum()

        # Preenche os valores nulos de paradas
        df_joined = self.__fill_occ(df_joined)

        # Recalcula os tempos
        df_joined = self.__group_and_calc_time(df_joined)

        # Se o tempo for negativo, ajustar para 0
        df_joined["tempo"] = df_joined["tempo"].clip(lower=0)

        # Se o tempo for maior que 480 minutos, ajustar para 480
        df_joined["tempo"] = df_joined["tempo"].clip(upper=480)

        # Remove colunas auxiliares
        df_joined = df_joined.drop(
            columns=[
                "maquina_id_change",
                "change",
                "maquina_id_change",
                "change_date",
                "motivo_change",
                "group",
            ]
        )

        return df_joined

    def get_time_working(self, df: pd.DataFrame) -> pd.Series:
        """
        Calcula o tempo de trabalho de cada máquina.
        """

        # Filtra apenas as máquinas que estão rodando
        df = df[df["status"] == "rodando"]

        # Agrupa por máquina e turno
        df = (
            df.groupby(["maquina_id", "turno", "data_registro", "status"], observed=False)["tempo"]
            .sum()
            .reset_index()
        )

        # Renomear coluna
        df = df.rename(columns={"status": "motivo"})

        # Capitaliza o motivo
        df["motivo"] = df["motivo"].str.capitalize()

        return df

    def get_maq_stopped(self, df: pd.DataFrame) -> pd.Series:

        # Filtra apenas as máquinas que estão paradas
        df = df[df["status"] == "parada"]

        # Reinicia o índice
        df = df.reset_index(drop=True)

        return df


service_info_ihm = ServiceInfoIHM()

## Saídas do serviço


In [169]:
df_info_ihm = service_info_ihm.get_info_ihm_adjusted()
df_info_ihm

C:\Users\bruno.thomaz\AppData\Local\Temp\ipykernel_22404\866565707.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["motivo"] = df.groupby("group")["motivo"].transform(lambda x: x.ffill().bfill())
C:\Users\bruno.thomaz\AppData\Local\Temp\ipykernel_22404\866565707.py:52: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
C:\Users\bruno.thomaz\AppData\Local\Temp\ipykernel_22404\866565707.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call resu

,fabrica,linha,maquina_id,turno,status,data_registro,hora_registro,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro_ihm,hora_registro_ihm,data_hora,data_hora_final,tempo
0,1,1,TMF003,NOT,parada,2024-05-01,00:01:06,Parada Programada,None,Parada Planejada,Backup,None,002357,2024-05-01,06:47:36,2024-05-01 00:01:06,2024-05-01 08:01:08,480
1,1,1,TMF003,MAT,parada,2024-05-01,08:01:08,Parada Programada,None,Parada Planejada,Sem Produção,None,002357,2024-05-01,11:28:20,2024-05-01 08:01:08,2024-05-01 16:01:09,480
2,1,1,TMF003,VES,parada,2024-05-01,16:01:09,None,None,None,None,None,None,NaT,None,2024-05-01 16:01:09,2024-05-02 00:01:10,480
3,1,1,TMF003,NOT,parada,2024-05-02,00:01:10,None,None,None,None,None,None,NaT,None,2024-05-02 00:01:10,2024-05-02 00:21:11,20
4,1,1,TMF003,NOT,rodando,2024-05-02,00:21:11,None,None,None,None,None,None,NaT,None,2024-05-02 00:21:11,2024-05-02 00:33:11,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,2,13,TMF013,VES,parada,2024-05-02,16:00:10,None,None,None,None,None,None,NaT,None,2024-05-02 16:00:10,2024-05-03 00:00:37,480
465,2,14,TMF009,VES,parada,2024-05-01,22:00:08,None,None,None,None,None,None,NaT,None,2024-05-01 22:00:08,2024-05-02 00:00:08,120
466,2,14,TMF009,NOT,parada,2024-05-02,00:00:08,None,None,None,None,None,None,NaT,None,2024-05-02 00:00:08,2024-05-02 08:00:10,480
467,2,14,TMF009,MAT,parada,2024-05-02,08:00:10,None,None,None,None,None,None,NaT,None,2024-05-02 08:00:10,2024-05-02 16:00:11,480


In [170]:
df_time_working = service_info_ihm.get_time_working(df_info_ihm.copy())
df_time_working

,maquina_id,turno,data_registro,motivo,tempo
0,TMF001,MAT,2024-05-02,Rodando,400
1,TMF001,MAT,2024-05-03,Rodando,172
2,TMF001,NOT,2024-05-02,Rodando,194
3,TMF001,NOT,2024-05-03,Rodando,62
4,TMF001,VES,2024-05-02,Rodando,320
5,TMF002,MAT,2024-05-02,Rodando,458
6,TMF002,NOT,2024-05-02,Rodando,380
7,TMF002,NOT,2024-05-03,Rodando,276
8,TMF002,VES,2024-05-02,Rodando,346
9,TMF003,MAT,2024-05-02,Rodando,176


In [171]:
df_maq_stopped = service_info_ihm.get_maq_stopped(df_info_ihm.copy())
df_maq_stopped

,fabrica,linha,maquina_id,turno,status,data_registro,hora_registro,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro_ihm,hora_registro_ihm,data_hora,data_hora_final,tempo
0,1,1,TMF003,NOT,parada,2024-05-01,00:01:06,Parada Programada,None,Parada Planejada,Backup,None,002357,2024-05-01,06:47:36,2024-05-01 00:01:06,2024-05-01 08:01:08,480
1,1,1,TMF003,MAT,parada,2024-05-01,08:01:08,Parada Programada,None,Parada Planejada,Sem Produção,None,002357,2024-05-01,11:28:20,2024-05-01 08:01:08,2024-05-01 16:01:09,480
2,1,1,TMF003,VES,parada,2024-05-01,16:01:09,None,None,None,None,None,None,NaT,None,2024-05-01 16:01:09,2024-05-02 00:01:10,480
3,1,1,TMF003,NOT,parada,2024-05-02,00:01:10,None,None,None,None,None,None,NaT,None,2024-05-02 00:01:10,2024-05-02 00:21:11,20
4,1,1,TMF003,NOT,parada,2024-05-02,00:33:11,None,None,None,None,None,None,NaT,None,2024-05-02 00:33:11,2024-05-02 01:17:11,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2,13,TMF013,VES,parada,2024-05-02,16:00:10,None,None,None,None,None,None,NaT,None,2024-05-02 16:00:10,2024-05-03 00:00:37,480
249,2,14,TMF009,VES,parada,2024-05-01,22:00:08,None,None,None,None,None,None,NaT,None,2024-05-01 22:00:08,2024-05-02 00:00:08,120
250,2,14,TMF009,NOT,parada,2024-05-02,00:00:08,None,None,None,None,None,None,NaT,None,2024-05-02 00:00:08,2024-05-02 08:00:10,480
251,2,14,TMF009,MAT,parada,2024-05-02,08:00:10,None,None,None,None,None,None,NaT,None,2024-05-02 08:00:10,2024-05-02 16:00:11,480


# Dados de Saída


In [172]:
print(f"Maquina Parada --> Linhas x Colunas: {df_maq_stopped.shape}")
print("---------------------------------------")
print(
    f"Maquina Parada --> Memória Ocupada: {df_maq_stopped.memory_usage().sum() / 1024 ** 2:.2f} MB"
)
print("---------------------------------------")
print(f"Maquina Parada --> Tipos:\n{df_maq_stopped.dtypes}")

Maquina Parada --> Linhas x Colunas: (253, 18)
---------------------------------------
Maquina Parada --> Memória Ocupada: 0.03 MB
---------------------------------------
Maquina Parada --> Tipos:
fabrica                      object
linha                         int32
maquina_id                   object
turno                        object
status                       object
data_registro        datetime64[ns]
hora_registro                object
motivo                       object
equipamento                  object
problema                     object
causa                        object
os_numero                    object
operador_id                  object
data_registro_ihm    datetime64[ns]
hora_registro_ihm            object
data_hora            datetime64[ns]
data_hora_final      datetime64[ns]
tempo                         int32
dtype: object


In [173]:
print(f"Tempo de Trabalho --> Linhas x Colunas: {df_time_working.shape}")
print("---------------------------------------")
print(
    f"Tempo de Trabalho --> Memória Ocupada: {df_time_working.memory_usage().sum() / 1024 ** 2:.2f} MB"
)
print("---------------------------------------")
print(f"Tempo de Trabalho --> Tipos:\n{df_time_working.dtypes}")

Tempo de Trabalho --> Linhas x Colunas: (48, 5)
---------------------------------------
Tempo de Trabalho --> Memória Ocupada: 0.00 MB
---------------------------------------
Tempo de Trabalho --> Tipos:
maquina_id               object
turno                    object
data_registro    datetime64[ns]
motivo                   object
tempo                     int32
dtype: object


In [174]:
print(f"Informações --> Linhas x Colunas: {df_info_ihm.shape}")
print("---------------------------------------")
print(f"Informações --> Memória Ocupada: {df_info_ihm.memory_usage().sum() / 1024 ** 2:.2f} MB")
print("---------------------------------------")
print(f"Informações --> Tipos:\n{df_info_ihm.dtypes}")

Informações --> Linhas x Colunas: (469, 18)
---------------------------------------
Informações --> Memória Ocupada: 0.06 MB
---------------------------------------
Informações --> Tipos:
fabrica                      object
linha                         int32
maquina_id                   object
turno                        object
status                       object
data_registro        datetime64[ns]
hora_registro                object
motivo                       object
equipamento                  object
problema                     object
causa                        object
os_numero                    object
operador_id                  object
data_registro_ihm    datetime64[ns]
hora_registro_ihm            object
data_hora            datetime64[ns]
data_hora_final      datetime64[ns]
tempo                         int32
dtype: object


In [175]:
print(f"Produção --> Linhas x Colunas: {df_info_production_cleaned.shape}")
print("---------------------------------------")
print(
    f"Produção --> Memória Ocupada: {df_info_production_cleaned.memory_usage().sum() / 1024 ** 2:.2f} MB"
)
print("---------------------------------------")
print(f"Produção --> Tipos:\n{df_info_production_cleaned.dtypes}")

Produção --> Linhas x Colunas: (94, 8)
---------------------------------------
Produção --> Memória Ocupada: 0.01 MB
---------------------------------------
Produção --> Tipos:
fabrica                    object
linha                       int32
maquina_id                 object
turno                      object
total_ciclos              float64
total_produzido           float64
data_registro      datetime64[ns]
hora_registro              object
dtype: object


# Análise de Dados


## Eficiência, Performance e Reparos de Máquina/Linha


In [196]:
# cspell: words producao


class DataAnalysis:
    def __init__(self, df_stops: pd.DataFrame, df_prod: pd.DataFrame):
        self.df_stops = df_stops
        self.df_prod = df_prod

        # Dicionário com descontos de Eficiência
        self.desc_eff = {
            "Troca de Sabor": 15,
            "Troca de Produto": 35,
            "Refeição": 60,
            "Café e Ginástica Laboral": 10,
            "Treinamento": 60,
        }

        # Dicionário com descontos de Performance
        self.desc_perf = {
            "Troca de Sabor": 15,
            "Refeição": 60,
            "Café e Ginástica Laboral": 10,
            "Treinamento": 60,
        }

        # Dicionário com descontos de Reparo
        self.desc_rep = {"Troca de Produto": 35}

        # Dicionário com o que não afeta a eficiência
        self.not_eff = ["Sem Produção", "Backup"]

        # Dicionário com o que não afeta a performance
        self.not_perf = [
            "Sem Produção",
            "Backup",
            "Limpeza para parada de Fábrica",
            "Risco de Contaminação",
            "Parâmetros de Qualidade",
            "Manutenção",
        ]

        # Dicionário com o que afeta o reparo
        self.afeta_rep = ["Manutenção", "Troca de Produtos"]

    def __get_discount(
        self,
        df: pd.DataFrame,
        desc_dict: dict[str, int],
        not_dict: list[str],
        indicator: IndicatorType,
    ) -> pd.DataFrame:
        """
        Calcula o desconto de eficiência, performance ou reparo.
        """

        # Cria uma coluna com o desconto padrão
        df["desconto"] = 0

        # Aplica o desconto de acordo com as colunas "motivo" ou "problema" ou "causa"
        for key, value in desc_dict.items():
            mask = (
                df[["motivo", "problema", "causa"]]
                .apply(lambda x: x.str.contains(key, case=False, na=False))
                .any(axis=1)
            )
            df.loc[mask, "desconto"] = value

        # Caso o desconto seja maior que o tempo, o desconto deve ser igual ao tempo
        df["desconto"] = df[["desconto", "tempo"]].min(axis=1)

        # Caso o motivo, problema ou causa não afete o indicador, o desconto é igual a tempo
        mask = (
            df[["motivo", "problema", "causa"]]
            .apply(lambda x: x.str.contains("|".join(not_dict), case=False, na=False))
            .any(axis=1)
        )
        df.loc[mask, "desconto"] = df["tempo"]

        # Calcula o excedente
        df["excedente"] = (df["tempo"] - df["desconto"]).clip(lower=0)

        # Cria um dict para indicadores
        indicator_dict = {
            IndicatorType.EFFICIENCY: df,
            IndicatorType.PERFORMANCE: df[~mask],
            IndicatorType.REPAIR: df[mask],
        }

        return indicator_dict[indicator]

    def __get_elapsed_time(self, turno: str) -> int:
        """
        Calcula o tempo decorrido do turno.
        """

        # Agora
        now = datetime.now()

        if turno == "MAT" and 8 <= now.hour < 16:
            elapsed_time = now - datetime(now.year, now.month, now.day, 8, 0, 0)
        elif turno == "VES" and 16 <= now.hour < 24:
            elapsed_time = now - datetime(now.year, now.month, now.day, 16, 0, 0)
        elif turno == "NOT" and 0 <= now.hour < 8:
            elapsed_time = now - datetime(now.year, now.month, now.day, 0, 0, 0)
        else:
            return 480

        return elapsed_time.total_seconds() / 60

    def __get_expected_production_time(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calcula o tempo esperado de produção.
        """

        df["tempo_esperado"] = df.apply(
            lambda row: (
                np.floor(self.__get_elapsed_time(row["turno"]) - row["desconto"])
                if row["data_registro"].date() == pd.to_datetime("today").date()
                else 480 - row["desconto"]
            ),
            axis=1,
        )

        return df

    def get_eff_data(self) -> pd.DataFrame:
        """
        Calcula o desconto de eficiência.
        """

        df = self.df_stops
        df_prod = self.df_prod

        # Calcula o desconto de eficiência
        df = self.__get_discount(df, self.desc_eff, self.not_eff, IndicatorType.EFFICIENCY)
        ciclo_ideal = 10.6

        # Agrupa para ter o valor total de desconto
        df = (
            df.groupby(["maquina_id", "linha", "data_registro", "turno"], observed=False)
            .agg(
                tempo=("tempo", "sum"),
                desconto=("desconto", "sum"),
                excedente=("excedente", "sum"),
            )
            .reset_index()
        )

        # Une os dois dataframes
        df = pd.merge(df_prod, df, on=["maquina_id", "linha", "data_registro", "turno"], how="left")

        # Lida com valores nulos
        df = df.fillna(0)

        # Nova coluna para tempo esperado de produção
        df = self.__get_expected_production_time(df)

        # Nova coluna para produção esperada
        df["producao_esperada"] = (df["tempo_esperado"] * ciclo_ideal) * 2

        # Nova coluna para eficiência
        df["eficiencia"] = (df["total_produzido"] / df["producao_esperada"]).round(2)

        # Corrige valores nulos ou inf de eficiência
        df["eficiencia"] = df["eficiencia"].replace([np.inf, -np.inf], np.nan).fillna(0)

        # Se a produção esperada for 0 e a eficiência for 0, tornar a eficiência np.nan
        mask = (df["producao_esperada"] == 0) & (df["eficiencia"] == 0)
        df.loc[mask, "eficiencia"] = np.nan

        # Ordenar as colunas
        df = df[
            [
                "fabrica",
                "linha",
                "maquina_id",
                "turno",
                "data_registro",
                "hora_registro",
                "tempo",
                "total_produzido",
                "producao_esperada",
                "tempo_esperado",
                "desconto",
                "excedente",
                "eficiencia",
            ]
        ]

        return df

    def get_perf_data(self) -> pd.DataFrame:
        """
        Calcula o desconto de performance.
        """

        df = self.df_stops
        df_prod = self.df_prod

        # Dataframe com datas e turnos onde a causa está em not_eff e o tempo é igual a 480
        mask = (df["causa"].isin(self.not_perf)) & (df["tempo"] == 480)
        paradas_programadas = df[mask][["data_registro", "turno"]]  # Para performance ser np.nan

        # Calcula o desconto de performance e filtra para remover linhas que não afetam a performance
        df = self.__get_discount(df, self.desc_perf, self.not_perf, IndicatorType.PERFORMANCE)

        # Agrupa para ter o valor total de desconto
        df = (
            df.groupby(["maquina_id", "linha", "data_registro", "turno"], observed=False)
            .agg(
                tempo=("tempo", "sum"),
                desconto=("desconto", "sum"),
                afeta=("excedente", "sum"),
            )
            .reset_index()
        )

        # Une os dois dataframes
        df = pd.merge(df_prod, df, on=["maquina_id", "linha", "data_registro", "turno"], how="left")

        # Lida com valores nulos
        df = df.fillna(0)

        # Coluna com tempo esperado de produção
        df = self.__get_expected_production_time(df)

        # Coluna de Performance
        df["performance"] = df["afeta"] / df["tempo_esperado"]

        # Corrige valores nulos ou inf de performance
        df["performance"] = df["performance"].replace([np.inf, -np.inf], np.nan).fillna(0)

        # Se a data e turno estiverem em paradas_programadas, performance é np.nan e o tempo esperado é igual a 0
        mask = df["data_registro"].isin(paradas_programadas["data_registro"]) & df["turno"].isin(
            paradas_programadas["turno"]
        )
        df.loc[mask, "performance"] = np.nan
        df.loc[mask, "tempo_esperado"] = 0

        return df


data_analysis = DataAnalysis(df_maq_stopped.copy(), df_info_production_cleaned.copy())

## Saídas de Eficiência, Performance e Reparos de Máquina/Linha


In [197]:
df_eff = data_analysis.get_eff_data()
df_eff

,fabrica,linha,maquina_id,turno,data_registro,hora_registro,tempo,total_produzido,producao_esperada,tempo_esperado,desconto,excedente,eficiencia
0,1,1,TMF003,NOT,2024-05-01,07:59:08,480.0,12.0,0.0,0.0,480.0,0.0,NaN
1,1,1,TMF003,MAT,2024-05-01,15:59:09,480.0,12.0,0.0,0.0,480.0,0.0,NaN
2,1,1,TMF003,VES,2024-05-01,23:59:10,480.0,56.0,10176.0,480.0,0.0,480.0,0.01
3,1,1,TMF003,NOT,2024-05-02,07:59:12,176.0,6274.0,10176.0,480.0,0.0,176.0,0.62
4,1,1,TMF003,MAT,2024-05-02,15:59:13,304.0,2116.0,10176.0,480.0,0.0,304.0,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2,14,TMF009,NOT,2024-05-02,07:58:10,480.0,0.0,10176.0,480.0,0.0,480.0,0.00
90,2,14,TMF009,MAT,2024-05-02,15:58:11,480.0,0.0,10176.0,480.0,0.0,480.0,0.00
91,2,14,TMF009,VES,2024-05-02,23:58:38,480.0,0.0,10176.0,480.0,0.0,480.0,0.00
92,2,14,TMF009,NOT,2024-05-03,07:58:40,0.0,0.0,10176.0,480.0,0.0,0.0,0.00


In [198]:
df_perf = data_analysis.get_perf_data()
df_perf

,fabrica,linha,maquina_id,turno,total_ciclos,total_produzido,data_registro,hora_registro,tempo,desconto,afeta,tempo_esperado,performance
0,1,1,TMF003,NOT,18.0,12.0,2024-05-01,07:59:08,0.0,0.0,0.0,0.0,NaN
1,1,1,TMF003,MAT,18.0,12.0,2024-05-01,15:59:09,0.0,0.0,0.0,0.0,NaN
2,1,1,TMF003,VES,72.0,56.0,2024-05-01,23:59:10,480.0,0.0,480.0,480.0,1.000000
3,1,1,TMF003,NOT,6398.0,6274.0,2024-05-02,07:59:12,176.0,0.0,176.0,480.0,0.366667
4,1,1,TMF003,MAT,2124.0,2116.0,2024-05-02,15:59:13,304.0,0.0,304.0,480.0,0.633333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2,14,TMF009,NOT,0.0,0.0,2024-05-02,07:58:10,480.0,0.0,480.0,480.0,1.000000
90,2,14,TMF009,MAT,0.0,0.0,2024-05-02,15:58:11,480.0,0.0,480.0,480.0,1.000000
91,2,14,TMF009,VES,0.0,0.0,2024-05-02,23:58:38,480.0,0.0,480.0,480.0,1.000000
92,2,14,TMF009,NOT,0.0,0.0,2024-05-03,07:58:40,0.0,0.0,0.0,480.0,0.000000
